In [1]:
#import pymssql as sql
import pymysql as sql

In [7]:
class cook_object:

    def __init__(self):

        self.db_name = 'MICO_96'
        self.db_server = '10.2.4.146'
        self.db_login = 'asterisk'
        with open('sql.pass','r') as file:
            self.db_pass = file.read().replace('\n', '')
            file.close()
        self.con = self.connect_sql()

    def connect_sql(self):

        return sql.connect(
            self.db_server, 
            self.db_login, 
            self.db_pass,
            self.db_name
        )

In [8]:
cook = cook_object()

In [72]:
with cook.con:
    # drop temporary tables
    query = "drop table if exists primary_linkedid_0;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists primary_linkedid_1;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists primary_linkedid_2;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists linkedid_relation_batch_1_instance_0;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists linkedid_relation_batch_1_instance_1;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists linkedid_relation_batch_1_instance_2;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists linkedid_relation_batch_2_instance_0;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = "drop table if exists linkedid_related;"
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    # create known linkedid list
    query = """create temporary table primary_linkedid_0
    select
    'c1' as call_id,
    '1470142627.1800' as linkedid
    union all
    select
    'c2',
    '1470142698.1814';"""    
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    # duplicate
    query = """create temporary table primary_linkedid_1
    select
    call_id,
    linkedid
    from primary_linkedid_0;"""    
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = """create temporary table primary_linkedid_2
    select
    call_id,
    linkedid
    from primary_linkedid_0;"""
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    # union -> relation batch 1
    query = """create temporary table linkedid_relation_batch_1_instance_0
    select 
    primary_linkedid_0.call_id,
    primary_linkedid_0.linkedid,
    0 as relation_level
    from primary_linkedid_0 as primary_linkedid_0
    union all
    
    select
    primary_linkedid_1.call_id,
    relationsCalls.linkedid1,
    1
    from primary_linkedid_1 as primary_linkedid_1
    inner join relationsCalls 
    on primary_linkedid_1.linkedid=relationsCalls.linkedid2
    union all
    
    select
    primary_linkedid_2.call_id,
    relationsCalls.linkedid2,
    1    
    from primary_linkedid_2 as primary_linkedid_2
    inner join relationsCalls 
    on primary_linkedid_2.linkedid=relationsCalls.linkedid1;
    """
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    
    # duplicate relation batch 1
    query = """create temporary table linkedid_relation_batch_1_instance_1
    select 
    call_id,
    linkedid,
    relation_level
    from linkedid_relation_batch_1_instance_0;"""
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = """create temporary table linkedid_relation_batch_1_instance_2
    select 
    call_id,
    linkedid,
    relation_level
    from linkedid_relation_batch_1_instance_0;"""
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    # union -> relation batch 2
    query = """create temporary table linkedid_relation_batch_2_instance_0    
    select 
    linkedid_relation_batch_1_instance_0.call_id,
    linkedid_relation_batch_1_instance_0.linkedid
    from linkedid_relation_batch_1_instance_0 as linkedid_relation_batch_1_instance_0    
    union all
    
    select 
    linkedid_relation_batch_1_instance_1.call_id,
    relationsCalls.linkedid2
    from linkedid_relation_batch_1_instance_1 as linkedid_relation_batch_1_instance_1
    inner join relationsCalls 
    on linkedid_relation_batch_1_instance_1.linkedid=relationsCalls.linkedid1
    where 
    linkedid_relation_batch_1_instance_1.relation_level = 1
    union all
    
    select 
    linkedid_relation_batch_1_instance_2.call_id,
    relationsCalls.linkedid1
    from linkedid_relation_batch_1_instance_2 as linkedid_relation_batch_1_instance_2
    inner join relationsCalls 
    on linkedid_relation_batch_1_instance_2.linkedid=relationsCalls.linkedid2
    where 
    linkedid_relation_batch_1_instance_2.relation_level = 1;
    """
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    # group related linkedid
    query = """create temporary table linkedid_related
    select distinct
    call_id,
    linkedid COLLATE utf8_unicode_ci as linkedid
    from linkedid_relation_batch_2_instance_0"""
    cursor = cook.con.cursor()
    cursor.execute(query)
    
    query = """
    select
    linkedid_related.call_id,
    linkedid_related.linkedid,
    PT1C_cdr.recordingfile
    from PT1C_cdr as PT1C_cdr
    inner join linkedid_related as linkedid_related     
    on PT1C_cdr.linkedid = linkedid_related.linkedid"""
    
    query = """
    select    
    PT1C_cdr.recordingfile
    from PT1C_cdr as PT1C_cdr    
    where 
    PT1C_cdr.linkedid = '1602791299.14400266' or 
    PT1C_cdr.uniqueid = '1602791299.14400266'"""
    
    cursor = cook.con.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print(row)

In [77]:
with cook.con:
    query = """
    select
    PT1C_cdr.recordingfile,
    PT1C_cdr.linkedid,
    PT1C_cdr.uniqueid
    from PT1C_cdr as PT1C_cdr
    where 
    PT1C_cdr.linkedid = '"""+str(1602791299.14400266)+"""' or 
    PT1C_cdr.uniqueid = '"""+str(1602791299.14400266)+"""';"""

    cursor = cook.con.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print(row)

In [37]:
with cook.con:
    #query = "select * from PT1C_cdr limit 1;"
    #query = "select linkedid1,linkedid2,calldate from relationsCalls limit 3;"
    #query = "select count(calldate) from relationsCalls where calldate<'2020-01-01 00:00:00';"
    #query = "SHOW COLUMNS FROM PT1C_cdr;"
    
    cursor = cook.con.cursor()
    cursor.execute(query)
    for row in cursor.fetchall():
        print(row)

('id', 'int(11) unsigned', 'NO', 'PRI', None, 'auto_increment')
('calldate', 'datetime', 'NO', 'MUL', '0000-00-00 00:00:00', '')
('clid', 'varchar(80)', 'NO', 'MUL', '', '')
('src', 'varchar(80)', 'NO', 'MUL', '', '')
('dst', 'varchar(80)', 'NO', 'MUL', '', '')
('dcontext', 'varchar(80)', 'NO', 'MUL', '', '')
('lastapp', 'varchar(200)', 'NO', '', '', '')
('lastdata', 'varchar(200)', 'NO', '', '', '')
('duration', 'float unsigned', 'YES', '', None, '')
('billsec', 'float unsigned', 'YES', '', None, '')
('disposition', "enum('ANSWERED','BUSY','FAILED','NO ANSWER','CONGESTION')", 'YES', '', None, '')
('channel', 'varchar(50)', 'YES', '', None, '')
('dstchannel', 'varchar(50)', 'YES', '', None, '')
('amaflags', 'varchar(50)', 'YES', '', None, '')
('accountcode', 'varchar(20)', 'YES', '', None, '')
('uniqueid', 'varchar(32)', 'NO', '', '', '')
('userfield', 'varchar(200)', 'NO', '', '', '')
('answer', 'datetime', 'NO', '', None, '')
('end', 'datetime', 'NO', '', None, '')
('recordingfile', 